# 使用vLLM部署


# 一、安装相关依赖

在开始之前，请确保已将所需的依赖项安装到系统上：
```bash
pip install git+https://github.com/huggingface/transformers@d40f54fc2f1524458669048cb40a8d0286f5d1d2
pip install accelerate
pip install qwen-omni-utils
```


安装vllm
使用vLLM进行Qwen2.5-Omni的快速部署和推理时，需要从官方提供的源码构建vLLM以获取对Qwen2.5-Omni支持
```bash
git clone -b qwen2_omni_public_v1 https://github.com/fyabc/vllm.git
cd vllm
pip install .
```

# 二、下载模型文件
我们将模型下载到本地，方便离线推理;用modelscope库的snapshot_download函数下载模型。 
Qwen/Qwen2.5-Omni-7B是模型在平台上的名字， cache_dir是你要把模型存到哪里，你可以自己改，revision='master’表示下最新版本。

下载的时候，网络要稳定，存储路径别写错，不然就下不了。代码写好后，直接执行，模型就开始下载了，下载时间要看网速。 
注意：也可以直接将上面代码封装到python文件中（例如：download.py）直接执行python download.py即可。

In [ ]:
from modelscope import snapshot_download
model_dir = snapshot_download('Qwen/Qwen2.5-Omni-7B', cache_dir='/root/autodl-tmp', revision='master')

# 二、本地离线推理
使用vLLM来本地离线推理Qwen2.5-Omni，目前官方只支持vllm的thinker部分，因此输出的模型只能是文本。官方将在不久的未来支持模型的其他部分以实现音频输出。

In [ ]:
import os
import torch

from transformers import Qwen2_5OmniProcessor  # 导入用于处理多模态数据的处理器
from vllm import LLM, SamplingParams  # 导入 vLLM 模型和采样参数类
from qwen_omni_utils import process_mm_info  # 导入用于处理多模态信息的工具函数

# 禁用 vLLM engine v1（目前不支持）
os.environ['VLLM_USE_V1'] = '0'

# 模型路径，指向预训练模型的存储位置
MODEL_PATH = "/root/autodl-tmp/Qwen/Qwen2.5-Omni-7B"

# 初始化 LLM 模型
llm = LLM(
    model=MODEL_PATH,  # 指定模型路径
    trust_remote_code=True,  # 允许加载远程代码（例如模型配置）
    gpu_memory_utilization=0.9,  # 设置 GPU 内存利用率（0.9 表示使用 90% 的 GPU 内存）
    tensor_parallel_size=torch.cuda.device_count(),  # 设置张量并行的设备数量（根据 GPU 数量自动调整）
    limit_mm_per_prompt={'image': 1, 'video': 1, 'audio': 1},  # 限制每条提示中每种模态的最大数量（图像、视频、音频各 1 个）
    seed=1234,  # 设置随机种子以确保结果可复现
)

# 定义采样参数
sampling_params = SamplingParams(
    temperature=1e-6,  # 设置温度参数（接近 0 表示更确定性的输出）
    max_tokens=512,  # 设置生成的最大 token 数量
)

# 初始化多模态处理器
processor = Qwen2_5OmniProcessor.from_pretrained(MODEL_PATH)

# 定义对话消息，包括系统角色和用户角色
messages = [
    {
        "role": "system",
        "content": "You are Qwen, a virtual human developed by the Qwen Team, Alibaba Group, capable of perceiving auditory and visual inputs, as well as generating text and speech.",
    },  # 系统角色的描述，定义模型的行为和能力
    {
        "role": "user",
        "content": [
            {"type": "video", "video": "/root/autodl-tmp/draw.mp4"},
        ],
    },  # 用户角色的消息，包含一个视频文件路径
]

# 使用处理器将对话消息转换为模型输入的文本模板
# 并添加生成提示
text = processor.apply_chat_template(
    messages,
    tokenize=False,  # 不进行 token 化（直接使用文本）
    add_generation_prompt=True,  # 添加生成提示
)

# 使用工具函数处理多模态信息
# 提取音频、图像和视频数据
# 参数 use_audio_in_video=True 表示从视频中提取音频
audios, images, videos = process_mm_info(messages, use_audio_in_video=True)

# 构建模型输入字典
inputs = {
    'prompt': text[0],  # 文本提示
    'multi_modal_data': {},  # 多模态数据
    "mm_processor_kwargs": {
        "use_audio_in_video": True,  # 传递多模态处理器的参数
    },
}

# 根据提取的多模态数据，填充输入字典
if images is not None:
    inputs['multi_modal_data']['image'] = images  # 添加图像数据
if videos is not None:
    inputs['multi_modal_data']['video'] = videos  # 添加视频数据
if audios is not None:
    inputs['multi_modal_data']['audio'] = audios  # 添加音频数据

# 使用 LLM 模型生成输出
outputs = llm.generate(inputs, sampling_params=sampling_params)

# 打印生成的文本结果
print(outputs[0].outputs[0].text)